In [1]:
import pandas as pd
import openpyxl
import os

In [60]:
base_pagos_path = "Base Pagos 12.08.2024.xlsx"
asignacion_path = "base_asignacion_20240805.xlsx"
monoproducto = "monoproducto/MONOPRODUCTO.xlsx"
multiproducto = "multiproducto/MULTIPRODUCTO.xlsx"

In [44]:
df_base = pd.read_excel(base_pagos_path)
print(df_base.shape)
print(df_base.dtypes)
df_base.head()

(369, 9)
Entidad         int64
FECHA          object
CODCEN          int64
CLAVSERV        int64
CENTROPAGO      int64
IMPORTE       float64
MONEDA         object
NOMBRE         object
ESTADO         object
dtype: object


,Entidad,FECHA,CODCEN,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO
0,11,2024-08-12,23348892,248,879,32945.55,PEN,BRAVO'S SECURITY SAC,P
1,11,2024-08-12,30292656,248,232,1000.00,PEN,DNI 40166425 - 001102329600544135,P
2,11,2024-08-12,25390085,248,352,1238.00,PEN,.,P
3,11,2024-08-12,30053800,248,339,2000.00,PEN,ACEROS RAMS SAC,P
4,11,2024-08-12,24529616,248,240,70.00,PEN,ADA ROSA LOAYZA YARASCA,P


In [45]:
df_base['CODCEN'] = df_base['CODCEN'].astype(str).str.replace(' ', '').str.replace(r'\D', '', regex=True).str[-8:]
df_base['CODCEN'] = df_base['CODCEN'].str.zfill(8)
df_base['CODCEN'] = df_base['CODCEN'].astype(int)

In [46]:
df_base.isnull().sum()

Entidad       0
FECHA         0
CODCEN        0
CLAVSERV      0
CENTROPAGO    0
IMPORTE       0
MONEDA        0
NOMBRE        0
ESTADO        0
dtype: int64

In [47]:
df_asignacion = pd.read_excel(asignacion_path)
print(df_asignacion.shape)
print(df_asignacion.dtypes)
df_asignacion.head()

(242511, 40)
codigo                        int64
contrato                      int64
cont_18                       int64
nombre                       object
clave                        object
producto                     object
DiasVenc                      int64
moneda                       object
capital                     float64
liqtotal                    float64
agencia                      object
Camp                        float64
Gestor                       object
numdoc                       object
oficina                      object
territorio                   object
Tramo                        object
codoficina                    int64
CAPITAL_SOLES               float64
IMP_CAMPAÑA                 float64
TIPO_FONDO                   object
FLAG_HONRADO                  int64
TIPO_CARTERA                 object
DESC_TRAMO                   object
Hibrido                      object
ESTUDIO ABOGADO              object
FECHA_MORA                   object
ID_CASTIGO     

,codigo,contrato,cont_18,nombre,clave,producto,DiasVenc,moneda,capital,liqtotal,...,ID_CONDONACION MORA_TEMP,ID_FRACCIONADO,ID_KOBSA_P.CANCELACION,ID_REPORTE_CANC,ID_CARTERA,ID_FLAG,OBSERVACION,AGENCIA CORRECTA,PAG LEGAL,PAG. INTER
0,23493072,110005669600047145,1100059600047145,ROSEMARY HERNANDEZ DIAZ,BC,PRÉSTAMOS COMERCIALES,838,PEN,4557.98,16173.88,...,-,-,-,-,STOCK,4,-,CLASA MORA,-,-
1,28140429,110010635000478837,1100105000478837,CLAUDIO ROJAS ANDRADA,BC,TARJETAS PYMES,246,PEN,89330.16,133945.59,...,-,1,-,-,STOCK,1,-,CLASA MORA,-,-
2,30739481,110010629600055161,1100109600055161,DINA CHOQUE FERNANDEZ,BC,PRÉSTAMOS PYMES,257,PEN,76663.97,108418.69,...,-,-,-,-,STOCK,2,PAGO LEGAL AGOSTO,MORNESE MORA,1,-
3,30739481,110010619600057679,1100109600057679,DINA CHOQUE FERNANDEZ,BC,PRÉSTAMOS PYMES,268,PEN,177075.30,256680.65,...,-,-,-,-,STOCK,2,PAGO LEGAL AGOSTO,MORNESE MORA,1,-
4,21874900,110050295000360222,1100505000360222,JULIA ANGELICA SOTO COTRINA,BC,TARJETAS PYMES,307,PEN,23986.04,36966.81,...,-,-,-,-,STOCK,4,PAGO INTERBANCARIO,ASESCOM RJ,-,1


In [48]:
###############################################################################################################################################################

In [49]:
df_mono = df_base.copy()
print(df_mono.shape)

(369, 9)


In [50]:
df_asignacion_test = df_asignacion.copy()
print(df_asignacion_test.shape)

(242511, 40)


In [51]:
df_mono = df_mono.merge(df_asignacion_test[['codigo', 'ID_FLAG', 'cont_18', 'TIPO_CARTERA', 'TIPO_FONDO']], left_on='CODCEN', right_on='codigo', how='left')
df_mono = df_mono.drop(columns=['codigo'])
df_mono = df_mono.rename(columns={'ID_FLAG': 'FLAG', 'cont_18': 'CONTRATO'})
print(df_mono.shape)
df_mono.head()

(459, 13)


,Entidad,FECHA,CODCEN,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_CARTERA,TIPO_FONDO
0,11,2024-08-12,23348892,248,879,32945.55,PEN,BRAVO'S SECURITY SAC,P,1.0,1.101790e+15,UNSECURED,REACTIVA
1,11,2024-08-12,30292656,248,232,1000.00,PEN,DNI 40166425 - 001102329600544135,P,1.0,1.102330e+15,UNSECURED,NaN
2,11,2024-08-12,25390085,248,352,1238.00,PEN,.,P,1.0,1.101760e+15,UNSECURED,NaN
3,11,2024-08-12,30053800,248,339,2000.00,PEN,ACEROS RAMS SAC,P,1.0,1.103400e+15,UNSECURED,NaN
4,11,2024-08-12,24529616,248,240,70.00,PEN,ADA ROSA LOAYZA YARASCA,P,1.0,1.108315e+15,UNSECURED,NaN


In [52]:
df_mono["FLAG"] = df_mono["FLAG"].astype("Int64")
df_mono["CONTRATO"] = df_mono["CONTRATO"].astype("Int64")
df_mono.dtypes

Entidad           int64
FECHA            object
CODCEN            int32
CLAVSERV          int64
CENTROPAGO        int64
IMPORTE         float64
MONEDA           object
NOMBRE           object
ESTADO           object
FLAG              Int64
CONTRATO          Int64
TIPO_CARTERA     object
TIPO_FONDO       object
dtype: object

In [53]:
df_multi = df_mono[df_mono["FLAG"] > 1].copy()
print(df_multi.shape)
df_multi.head()

(145, 13)


,Entidad,FECHA,CODCEN,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_CARTERA,TIPO_FONDO
6,11,2024-08-10,20915360,248,135,150.0,PEN,AGRIPINO BARRIENTOOS HUARCAYA,P,4,1108149600611227,UNSECURED,NaN
7,11,2024-08-10,20915360,248,135,150.0,PEN,AGRIPINO BARRIENTOOS HUARCAYA,P,4,1108149600635142,UNSECURED,NaN
8,11,2024-08-10,20915360,248,135,150.0,PEN,AGRIPINO BARRIENTOOS HUARCAYA,P,4,1108149600628367,UNSECURED,NaN
9,11,2024-08-10,20915360,248,135,150.0,PEN,AGRIPINO BARRIENTOOS HUARCAYA,P,4,1108149600640715,UNSECURED,NaN
10,11,2024-08-10,20915360,248,135,150.0,PEN,AGRIPINO BARRIENTOS HUARCAYA,P,4,1108149600611227,UNSECURED,NaN


In [54]:
df_mono = df_mono[df_mono["FLAG"] == 1]
print(df_mono.shape)
df_mono.head()

(309, 13)


,Entidad,FECHA,CODCEN,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_CARTERA,TIPO_FONDO
0,11,2024-08-12,23348892,248,879,32945.55,PEN,BRAVO'S SECURITY SAC,P,1,1101789600299189,UNSECURED,REACTIVA
1,11,2024-08-12,30292656,248,232,1000.00,PEN,DNI 40166425 - 001102329600544135,P,1,1102329600544135,UNSECURED,NaN
2,11,2024-08-12,25390085,248,352,1238.00,PEN,.,P,1,1101759600282565,UNSECURED,NaN
3,11,2024-08-12,30053800,248,339,2000.00,PEN,ACEROS RAMS SAC,P,1,1103399600153227,UNSECURED,NaN
4,11,2024-08-12,24529616,248,240,70.00,PEN,ADA ROSA LOAYZA YARASCA,P,1,1108315001994824,UNSECURED,NaN


In [55]:
df_mono.fillna({"TIPO_CARTERA": "NULL"}, inplace=True)
df_mono["TIPO_CARTERA"].value_counts()

TIPO_CARTERA
UNSECURED    306
SECURED        2
NULL           1
Name: count, dtype: int64

In [56]:
df_mono.fillna({"TIPO_FONDO": "NULL"}, inplace=True)
df_mono["TIPO_FONDO"].value_counts()

TIPO_FONDO
NULL        307
REACTIVA      2
Name: count, dtype: int64

In [57]:
df_mono[["TIPO_CARTERA", "TIPO_FONDO"]].value_counts()

TIPO_CARTERA  TIPO_FONDO
UNSECURED     NULL          304
SECURED       NULL            2
UNSECURED     REACTIVA        2
NULL          NULL            1
Name: count, dtype: int64

In [58]:
df_mono = df_mono[df_mono["TIPO_CARTERA"] == "UNSECURED"]
df_mono = df_mono[df_mono["TIPO_FONDO"] == "NULL"]
print(df_mono.shape)

(304, 13)


In [61]:
df_mono["CONTRATO"] = df_mono["CONTRATO"].astype(str).str.zfill(18)

1      001102329600544135
2      001101759600282565
3      001103399600153227
4      001108315001994824
5      001105045000973108
              ...        
454    001101415002134828
455    001107855004407653
456    001101415002100559
457    001101415002127465
458    001104085010855411
Name: CONTRATO, Length: 304, dtype: object

In [73]:
df_mono.to_excel(monoproducto, index=False)
os.startfile(os.path.abspath(monoproducto))